In [396]:
import numpy as np
import os
import cv2
from glob import glob
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd
from IPython.display import display_html
from itertools import chain,cycle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [397]:
#Define Processing Functions Here

def wb(channel, perc = 5):
    mi, ma = (np.percentile(channel, perc), np.percentile(channel,100.0-perc))
    channel = np.uint8(np.clip((channel-mi)*255.0/(ma-mi+0.01), 0, 255))
    return channel

def process_img(img):
    
    yCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    lower_color = np.array([0, 137, 77])
    upper_color = np.array([255, 163, 120])
    color_mask = cv2.inRange(yCrCb, lower_color, upper_color)
    yCrCb[color_mask == 0] = 0
    mask = cv2.inRange(yCrCb, np.array([180, 0, 0]), np.array([255, 255, 255]))
    yCrCb[mask == 255] = 0
    return yCrCb

def process_chroma_balance(img):
    yCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    yCrCb[:,:,1] = wb(yCrCb[:, :, 1])
    yCrCb = cv2.cvtColor(yCrCb, cv2.COLOR_YCrCb2BGR)
    yCrCb[np.argmax(yCrCb, axis=2) == 0] = 0
    yCrCb[np.argmax(yCrCb, axis=2) == 1] = 0
    yCrCb[np.argmax(yCrCb, axis=2) == 2] = 255

    return yCrCb

def process_Canny(img):
    grey = np.array(img)
    if len(img.shape)==3:
        grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(grey, 0, 0.3, apertureSize=7)
    
    return edges

def process_Laplacian(img):
    grey = np.array(img)
    if len(img.shape)==3:
        grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Laplacian(grey, -1, ksize=5)
    
    return edges

def process_Sobel(img):
    grey = np.array(img)
    if len(img.shape)==3:
        grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Sobel(grey, -1, 1, 1)
    
    return edges

In [398]:
# Prepare results directory
results_dir = "./results"
for folder in glob("*", root_dir="samples/"):
    os.makedirs(f"{results_dir}/{folder}", exist_ok=True)

In [414]:
# Save results if you want to see what the processing produces
save_results = True
if save_results:
    for file in glob(f"samples/*/*"):
        img = cv2.imread(f"{file}")
        img = cv2.resize(img, (512, 512))
        # img = process_chroma_balance(img)
        img = cv2.GaussianBlur(img, (5, 5), sigmaX=9)
        img = process_Laplacian(img)
        img = cv2.threshold(img, 200, 255, cv2.THRESH_BINARY)[1]
        # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, np.ones((3, 3)), iterations=2)
        # img = process_Canny(img)
        file_name = file.split('\\')[-1]
        folder_name = file.split('\\')[-2]
        cv2.imwrite(f"{results_dir}/{folder_name}/{file_name}", img)

In [400]:
# Prepare Data for Classifier

classes = ['one', 'two_up', 'three', 'four', 'palm']
y = []
training_images = []

# Change Size of feature vector based on the processing functions you use
array = np.array([], dtype=np.float32).reshape(0, 1 + 512*512*1)
i=0
for folder in classes:
    for file in glob(f"samples/{folder}/*"):
        img = cv2.imread(f"{file}")
        img = cv2.resize(img, (512, 512))

        # Use Processing Functions Here
        # img = process_chroma_balance(img)
        edges = process_Canny(img)
        array = np.concatenate((array,[np.concatenate(([i],
                # Add flattened features here
                # img.flatten(),
                edges.flatten()

        ), axis=0)]), axis=0)
    i+=1
print(array.shape)
sample_dataset = array

(250, 262145)


In [401]:
# Classifier
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        # html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        # html_str+=f'<h2 style="text-align: center;">{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        # html_str+='</td></th>'
    display_html(html_str,raw=True)

def model_prediction(model, x_training, y_training, x_validation, y_validation, weights=None):
	model.fit(x_training, y_training, sample_weight=weights)
	prediction_training = np.array(np.round(model.predict(x_training)), dtype=np.int8)
	prediction_validation = np.array(np.round(model.predict(x_validation)), dtype=np.int8)
	acc_training = accuracy_score(y_training, prediction_training)
	acc_validation = accuracy_score(y_validation, prediction_validation)

	confusion_matrix_training = confusion_matrix(y_training, prediction_training)
	confusion_matrix_validation = confusion_matrix(y_validation, prediction_validation)
	# print("Confusion Matrix Training: \n", confusion_matrix_training)
	display_side_by_side(pd.DataFrame(confusion_matrix_training), pd.DataFrame(confusion_matrix_validation))
	# print("Confusion Matrix Validation: \n", confusion_matrix_validation)
	print("Accuracy Score Training: ", acc_training)
	print("Accuracy Score Validation: ",acc_validation)



train, test = train_test_split(sample_dataset, test_size=0.2)
X_train = train[:,1:]
Y_train = train[:,0]
X_test = test[:,1:]
Y_test = test[:,0]


pca = PCA(n_components=30).fit(sample_dataset[:,1:])
model = SVC(C=10)
model_prediction(model, pca.transform(X=X_train), Y_train, pca.transform(X=X_test), Y_test)

,0,1,2,3,4
0,38,0,0,0,1
1,0,37,1,1,0
2,1,0,38,0,0
3,0,1,0,40,1
4,0,0,0,0,41
,0,1,2,3,4
0,2,1,5,2,1
1,4,1,2,4,0
2,4,3,1,1,2
3,1,2,1,2,2


Accuracy Score Training:  0.97
Accuracy Score Validation:  0.2
